# Amtrak Thruway Buses

Amtrak Thruway Buses do not have a `shape_id`, which means can't get `route` line geom associated with it.

We can get those trips in `stops` and `stop_times`.

In [ ]:
import dask_geopandas as dg
import dask.dataframe as dd
import geopandas as gpd
import intake
import pandas as pd

from shared_utils import geography_utils, rt_dates, rt_utils

catalog = intake.open_catalog("../high_quality_transit_areas/*.yml")

SELECTED_DATE = rt_dates.DATES["sep2022"]
COMPILED_CACHED_VIEWS = f"{rt_utils.GCS_FILE_PATH}compiled_cached_views/"
ITP_ID = 13

In [ ]:
trips = dd.read_parquet(
    f"{COMPILED_CACHED_VIEWS}trips_{SELECTED_DATE}_all.parquet")

amtrak_trips = trips[trips.calitp_itp_id == ITP_ID].compute()

thruway_bus = "Amtrak Thruway Connecting Service"

amtrak_thruway = amtrak_trips[amtrak_trips.route_long_name==thruway_bus]

In [ ]:
amtrak_thruway.head(2)

In [ ]:
routelines = dg.read_parquet(
        f"{COMPILED_CACHED_VIEWS}routelines_{SELECTED_DATE}_all.parquet")

amtrak_routes = routelines[routelines.calitp_itp_id==ITP_ID].compute()

for col in ["trip_id", "trip_key", "route_id"]:
    print(f"column: {col}")
    subset = amtrak_routes[amtrak_routes[col].isin(amtrak_thruway[col])]
    display(subset)

In [ ]:
stop_times = dd.read_parquet(
        f"{COMPILED_CACHED_VIEWS}st_{SELECTED_DATE}_all.parquet")

amtrak_stop_times = stop_times[stop_times.calitp_itp_id==ITP_ID].compute()

In [ ]:
stops = dg.read_parquet(
        f"{COMPILED_CACHED_VIEWS}stops_{SELECTED_DATE}_all.parquet")

amtrak_stops = stops[stops.calitp_itp_id==ITP_ID].compute()

In [ ]:
ca = catalog.ca_boundary.read()

ca_stops = amtrak_stops.clip(ca)

In [ ]:
amtrak_thruway_stop_times = amtrak_stop_times[
    (amtrak_stop_times.trip_id.isin(amtrak_thruway.trip_id)) & 
    (amtrak_stop_times.stop_id.isin(ca_stops.stop_id))
]

In [ ]:
amtrak_thruway_stop_times.head(2)